In [5]:
import requests
import re

# 下载并保存文件
url = 'https://www.gutenberg.org/files/100/100-0.txt'
response = requests.get(url)

# 移除UTF-8 BOM
text = response.content.decode('utf-8-sig') # 可以注释掉这行代码，你会发现获得的文本含有一些乱码

with open('shakespeare.txt', 'w', encoding='utf-8') as file:
    file.write(text)

print("下载完成并保存为 shakespeare.txt")

下载完成并保存为 shakespeare.txt


In [7]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from transformers import GPT2Tokenizer
from transformer import Transformer

# 1. 加载数据
with open('shakespeare.txt', 'r', encoding='utf-8') as file:
    text = file.read()

# 2. 使用GPT2的分词器（或其他分词器）
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
tokens = tokenizer.encode(text)

# 3. 创建数据集
class ShakespeareDataset(Dataset):
    def __init__(self, tokens, seq_length):
        self.tokens = tokens
        self.seq_length = seq_length

    def __len__(self):
        return len(self.tokens) // self.seq_length

    def __getitem__(self, idx):
        start = idx * self.seq_length
        end = start + self.seq_length
        return torch.tensor(self.tokens[start:end], dtype=torch.long)

# 4. 创建数据加载器
seq_length = 128
dataset = ShakespeareDataset(tokens, seq_length)
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

# 5. 定义模型、损失函数和优化器
model = Transformer(vocab_size=tokenizer.vocab_size, d_model=512, num_heads=8, num_layers=2, d_ff=2048, max_seq_len=seq_length)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

c:\Users\29724\AppData\Local\Programs\Python\Python310\lib\site-packages\huggingface_hub\file_download.py:140: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\29724\.cache\huggingface\hub\models--gpt2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (1837925 > 1024). Running this sequence through the model will result in indexing errors


In [8]:
from tqdm import tqdm

# 6. 训练循环
for epoch in range(10):
    for batch in tqdm(dataloader, desc=f"Epoch {epoch+1}"):
        optimizer.zero_grad()
        output = model(batch[:, :-1], batch[:, 1:])
        loss = criterion(output.view(-1, tokenizer.vocab_size), batch[:, 1:].reshape(-1))
        loss.backward()
        optimizer.step()
    print(f"Epoch [{epoch+1}/10], Loss: {loss.item():.4f}")

Epoch 1:   0%|          | 2/449 [00:10<37:28,  5.03s/it]


KeyboardInterrupt: 